In [105]:
import configparser
import os
import sys
import glob
import multiprocessing
import numpy as np

def generate_blas_flags():
	if 'CONDA_PREFIX' in os.environ:
		ld_dir = os.path.join(os.getenv("CONDA_PREFIX"), "Library", "bin")
		mkt_rt_bins = glob.glob(os.path.join(ld_dir, "mkl_rt*"))
		blas_flags = np.__config__.blas_mkl_info["libraries"]
		for b in mkt_rt_bins:
			if b.endswith(".dll"):
				b = b[:-4]
			blas_flags = [f"-l{os.path.basename(b)}"]
		return blas_flags
	else:
		raise NameError("Conda not installed.")

def generate_ld_path_flags():
	if 'CONDA_PREFIX' in os.environ:
		ld_dir = os.path.join(os.getenv("CONDA_PREFIX"), "Library", "bin")
		ld_path_flags = [f"-L{ld_dir}"]
		return ld_path_flags
	else:
		raise NameError("Conda not installed.")
	
def init_aesararc():
	HOMEPATH = os.path.expanduser("~")
	aesararc_config_file = os.path.join(HOMEPATH, ".aesararc")
	aesararc_config = configparser.ConfigParser()
	aesararc_config.add_section('global')
	aesararc_config["global"] = {'device': 'cpu', 'floatX': 'float64'}
	aesararc_config.add_section('blas')
	ldflags = "".join(f"{ld_path} " for ld_path in generate_ld_path_flags())
	ldflags += "".join(f"{blas} " for blas in generate_blas_flags())
	aesararc_config["blas"]["ldflags"] = ldflags
	with open(aesararc_config_file, "w") as f:
			aesararc_config.write(f)

def _config():
	config = configparser.ConfigParser()
	config.add_section("hyperparameters")
	config["hyperparameters"] = {
		"python_version": sys.version,
		"cwd": os.getcwd(),
		"patience": 9000,
		"patience_increase": 2,
		"validation_threshold": 1.003,
		"seed": 999,
		"base_lr": 0.0001,
		"max_lr": 0.01,
		"max_epoch": 2000,
		"batch_size": 64,
		"cyclic_lr_mode": "triangular2",
		"cyclic_lr_step_size": 8,
	}
	return config

def init_env_vars():
	num_cores = multiprocessing.cpu_count()
	os.environ["MKL_NUM_THREADS"] = str(num_cores)
	os.environ["OMP_NUM_THREADS"] = str(num_cores)

class Config:
	def __init__(self):
		self.config = _config()
		init_aesararc()
		init_env_vars()

	def __repr__(self):
		rval = f"config ={{\n"
		for key, val in self.config["hyperparameters"].items():
			rval += (f"    {key}: {val},\n")
		rval += f"}}"
		return rval
	
	def __getitem__(self, name):
		if name in self.config["hyperparameters"]:
			return self.config["hyperparameters"][name]

	def __setitem__(self, name, val):
		if name in self.config["hyperparameters"]:
			self.config["hyperparameters"][name] = val
		else:
			raise NameError(f"{name} not found in config file.")

cfg = Config()
cfg["patience"]

'9000'

In [107]:
print(cfg)

config ={
    python_version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)],
    cwd: c:\Users\Melvin\OneDrive - TU Eindhoven\Lab\pycmtensor,
    patience: 9000,
    patience_increase: 2,
    validation_threshold: 1.003,
    seed: 999,
    base_lr: 0.0001,
    max_lr: 0.01,
    max_epoch: 2000,
    batch_size: 64,
    cyclic_lr_mode: triangular2,
    cyclic_lr_step_size: 8,
}
